In [1]:
# Standard libraries
import sys
import math
import random
import time
import os

# Third-party imports
import numpy as np
import pandas as pd
import pickle as pkl

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

import keras_tuner as kt

import importlib
from yapf.yapflib.yapf_api import FormatCode
import GPUtil

# Project imports
import ephesus
import utils
from const import *

#### GPU Setup

In [1]:
# Specific GPUs
# gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[1:4], 'GPU')
mirror_strat = tf.distribute.MirroredStrategy()

NameError: name 'tf' is not defined

In [2]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [3]:
import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.callbacks import TensorBoard
# from tensorflow.keras.callbacks import EarlyStopping

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')

In [5]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [5]:
# All GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print('here')

here


In [6]:
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
mirror_strat = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2022-03-19 22:01:09.336247: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-19 22:01:09.360181: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x55a16dc29630 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-03-19 22:01:09.360212: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): Host, Default Version
2022-03-19 22:01:09.950533: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x55a1690c8120 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-03-19 22:01:09.950574: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Ca

In [6]:
# All GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print('here')
    try:
        print('here')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)
mirror_strat = tf.distribute.MirroredStrategy()

here
here
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2022-03-19 22:06:23.245828: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-19 22:06:23.282356: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x55efd1a2b6d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-03-19 22:06:23.282395: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): Host, Default Version
2022-03-19 22:06:23.870981: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x55efd11b1b50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-03-19 22:06:23.871045: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Ca

#### Data Preprocessing

In [7]:
###      DOUBLE CHECK      ###
detec_type = 'plane_moon_cut_injec'

In [8]:
# Find the most recent file in injec_curve
datas = utils.retur_most_recen(f'{xom_data_path}padde_cuts/')
# datas = 'padde_cuts-89885-not_detre-separ_moon_signa-1644394304.pkl'
print(f'Most recent file: {datas}')

with open(f'{xom_data_path}padde_cuts/{datas}','rb') as f:
    raw_x_data = pkl.load(f)

Most recent file: padde_cuts-34925-1370-8598.pkl


In [9]:
numbe_sampl = -1

# Create a shuffled sample of the data
rando_shuff_x_data = raw_x_data[utils.retur_rando_sampl(
    numbe_sampl, len(raw_x_data))]
np.random.shuffle(rando_shuff_x_data)

# Normalize the data from 0 to 1
norma_full_x_data = np.copy(rando_shuff_x_data)
x_data = np.copy(rando_shuff_x_data[:, :-1, 1].astype(float))

for i in range(len(x_data)):
    chang_slots = np.where(x_data[i] != 0)[0]
    x_data[i, chang_slots] = utils.norma_data(x_data[i, chang_slots])
    norma_full_x_data[i, :-1, 1] = x_data[i]

x_data = np.expand_dims(x_data, 2)

# Create the corresponding y dataset
y_data = np.zeros(len(norma_full_x_data))
for i in range(len(norma_full_x_data)):
    y_data[i] = norma_full_x_data[i, -1, 1][detec_type]

print(
    f'{sum(y_data)/len(y_data):.2%} of the dataset contains a full injection')

19.15% of the dataset contains a full injection


In [10]:
# Split the data into training and testing

stop = len(x_data) - len(y_data) % 100
split = int(stop * 0.7)
    
x_train_np = np.array(x_data[:split])
full_x_train = norma_full_x_data[:split]
x_test_np = np.array(x_data[split:stop])
full_x_test = norma_full_x_data[split:stop]
y_train_np = y_data[:split]
y_test_np = y_data[split:stop]

In [11]:
# Mask the data
masking = layers.Masking(mask_value=0., input_shape = (1900, 1))

x_train_unmas = tf.convert_to_tensor(x_train_np)
x_test_unmas = tf.convert_to_tensor(x_test_np)
y_train = tf.convert_to_tensor(y_train_np)
y_test = tf.convert_to_tensor(y_test_np)
x_train = masking(x_train_unmas)
x_test = masking(x_test_unmas)

#### Training

In [12]:
# Callbacks

tensorboard = TensorBoard(log_dir=f'{main_path}tensorboard/{datas}_{int(time.time())}')
early_stopp = EarlyStopping(monitor='val_accuracy', patience=6)
resto_valid_accur = utils.resto_best_valid_accur()
email_train_progr = utils.email_train_progr()

callb = [resto_valid_accur, early_stopp, email_train_progr]#, tensorboard]

In [ ]:
utils.send_task_comple_email('KT')

In [15]:
model = tuner.get_best_models(num_models=3)

In [ ]:
model_num = 0
model_path = f'{xom_data_path}plane_moon_model/{datas}-{model_num}-{int(time.time())}.h5'
model[model_num].save(model_path)
print(f'Best Model Path: {model_path}')

In [17]:
model_num = 0
for model_num in range(len(model)):
    model_path = f'{xom_data_path}plane_moon_model/{datas}-{model_num}-{int(time.time())}.h5'
    model[model_num].save(model_path)
    print(f'Best Model Path: {model_path}')

Best Model Path: /data/scratch/data/moon/plane_moon_model/padde_cuts-not_detre--1642315002.pkl-0-1642476005.h5


## Testing

#### Formatting